In [1]:
%load_ext cython

In [2]:
import numpy as np

def load_profits_from_file(filename, num_objectives, num_items):
    profits = np.zeros((num_objectives, num_items))
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    idx = 0
    idx += 1  # skip header
    for obj in range(num_objectives):
        idx += 1  # skip capacity line
        for item in range(num_items):
            idx += 1  # skip "n:" line
            idx += 1  # skip weight
            profit = int(lines[idx])
            profits[obj, item] = profit
            idx += 1
    return profits

def load_weights_from_file(filename, num_objectives, num_items):
    weights = np.zeros((num_objectives, num_items), dtype=int)
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    idx = 0
    idx += 1  # skip header
    for obj in range(num_objectives):
        idx += 1  # skip capacity line
        for item in range(num_items):
            idx += 1  # skip "n:" line
            weight = int(lines[idx])
            weights[obj, item] = weight
            idx += 2  # skip profit
    return weights

def load_capacities_from_file(filename, num_objectives, num_items):
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    idx = 0
    idx += 1  # skip header
    capacities = []
    for obj in range(num_objectives):
        cap = float(lines[idx])
        capacities.append(cap)
        idx += 1 + 3 * num_items
    return np.array(capacities)

In [11]:
import numpy as np
import random
import time
import json
import os
from typing import List, Dict, Tuple, Optional
import matplotlib.pyplot as plt

# Optional LLM integration - falls back to built-in functions if not available
ENABLE_LLM = False
try:
    import requests
    OLLAMA_URL = "http://localhost:11434/api/generate"
    MODEL_NAME = "llama3"
    
    # Test if Ollama is available
    response = requests.get("http://localhost:11434/api/tags", timeout=2)
    if response.status_code == 200:
        ENABLE_LLM = True
        print("LLM integration enabled - Ollama detected")
    else:
        print("LLM integration disabled - Ollama not available")
except:
    print("LLM integration disabled - falling back to built-in functions")

# Mock LLM functions for when Ollama is not available
def generate_llm_response(system_prompt, user_prompt, max_new_tokens=500):
    """Mock LLM response - returns basic function templates"""
    if "mutation" in user_prompt.lower():
        return """
def mutation_move(observation, profits, weights, capacities, archive_objs=None, best_observation=None):
    items_selected = np.where(observation == 1)[0]
    items_unselected = np.where(observation == 0)[0]
    if len(items_selected) == 0 or len(items_unselected) == 0:
        return 0, 0
    remove_idx = int(np.random.choice(items_selected))
    add_idx = int(np.random.choice(items_unselected))
    return remove_idx, add_idx
"""
    elif "local_search" in user_prompt.lower():
        return """
def local_search_move(observation, profits, weights, capacities, archive_objs=None, best_observation=None):
    items_selected = np.where(observation == 1)[0]
    items_unselected = np.where(observation == 0)[0]
    if len(items_selected) == 0 or len(items_unselected) == 0 or profits is None:
        return 0, 0
    profs_selected = np.sum(profits[:, items_selected], axis=0)
    profs_unselected = np.sum(profits[:, items_unselected], axis=0)
    remove_idx = items_selected[np.argmin(profs_selected)]
    add_idx = items_unselected[np.argmax(profs_unselected)]
    return int(remove_idx), int(add_idx)
"""
    else:
        return """
def default_move(observation, profits, weights, capacities, archive_objs=None, best_observation=None):
    items_selected = np.where(observation == 1)[0]
    items_unselected = np.where(observation == 0)[0]
    if len(items_selected) == 0 or len(items_unselected) == 0:
        return 0, 0
    remove_idx = int(np.random.choice(items_selected))
    add_idx = int(np.random.choice(items_unselected))
    return remove_idx, add_idx
"""

def extract_code_from_response(response):
    """Extract Python code from LLM response."""
    markers = [
        ("```python", "```"),
        ("```Python", "```"),
        ("```", "```"),
        ("`", "`")
    ]
    
    for start_marker, end_marker in markers:
        start_idx = response.find(start_marker)
        if start_idx != -1:
            start_idx += len(start_marker)
            end_idx = response.find(end_marker, start_idx)
            if end_idx != -1:
                return response[start_idx:end_idx].strip()
    
    return response.strip()

def validate_function(code, function_name, input_args):
    """Validate if the generated function is syntactically correct and can be executed."""
    try:
        namespace = {'np': np, 'numpy': np}
        exec(code, namespace)
        
        if function_name not in namespace:
            return False, f"Function '{function_name}' not found in the generated code."
        
        func = namespace[function_name]
        dummy_inputs = []
        for arg in input_args:
            if isinstance(arg, tuple) and arg[0] == 'array':
                dummy_inputs.append(np.zeros(arg[1]))
            elif isinstance(arg, tuple) and arg[0] == 'array_2d':
                dummy_inputs.append(np.zeros(arg[1]))
            elif isinstance(arg, tuple) and arg[0] == 'dict':
                dummy_inputs.append({})
            elif isinstance(arg, tuple) and arg[0] == 'float':
                dummy_inputs.append(0.1)
            else:
                dummy_inputs.append(0)
        
        result = func(*dummy_inputs)
        
        if not isinstance(result, tuple) or len(result) != 2:
            return False, f"Function should return a tuple of two integers"
            
        remove_idx, add_idx = result
        if not isinstance(remove_idx, (int, np.integer)) or not isinstance(add_idx, (int, np.integer)):
            return False, f"Function should return integers"
        
        return True, "Function is valid."
    except Exception as e:
        return False, f"Error validating function: {str(e)}"

LLM integration enabled - Ollama detected


In [4]:
def mutation_move(observation, profits, weights, capacities, archive_objs=None, best_observation=None):
    items_selected = np.where(observation == 1)[0]
    items_unselected = np.where(observation == 0)[0]
    if len(items_selected) == 0 or len(items_unselected) == 0:
        return 0, 0
    remove_idx = int(np.random.choice(items_selected))
    add_idx = int(np.random.choice(items_unselected))
    return remove_idx, add_idx

def local_search_move(observation, profits, weights, capacities, archive_objs=None, best_observation=None):
    items_selected = np.where(observation == 1)[0]
    items_unselected = np.where(observation == 0)[0]
    if len(items_selected) == 0 or len(items_unselected) == 0 or profits is None:
        return 0, 0
    profs_selected = np.sum(profits[:, items_selected], axis=0)
    profs_unselected = np.sum(profits[:, items_unselected], axis=0)
    remove_idx = items_selected[np.argmin(profs_selected)]
    add_idx = items_unselected[np.argmax(profs_unselected)]
    return int(remove_idx), int(add_idx)

def global_search_move(observation, profits, weights, capacities, archive_objs=None, best_observation=None):
    items_selected = np.where(observation == 1)[0]
    items_unselected = np.where(observation == 0)[0]
    if len(items_selected) == 0 or len(items_unselected) == 0 or profits is None:
        return 0, 0
    profs_selected = np.sum(profits[:, items_selected], axis=0)
    profs_unselected = np.sum(profits[:, items_unselected], axis=0)
    ps = profs_selected + 1e-9
    pu = profs_unselected + 1e-9
    remove_idx = np.random.choice(items_selected, p=ps / np.sum(ps))
    add_idx = np.random.choice(items_unselected, p=pu / np.sum(pu))
    return int(remove_idx), int(add_idx)

def follow_move(observation, profits, weights, capacities, archive_objs=None, best_observation=None):
    if best_observation is None:
        return mutation_move(observation, profits, weights, capacities)
    diff = best_observation - observation
    add_candidates = np.where((diff == 1) & (observation == 0))[0]
    remove_candidates = np.where((diff == -1) & (observation == 1))[0]
    if len(add_candidates) == 0 or len(remove_candidates) == 0:
        return mutation_move(observation, profits, weights, capacities)
    add_idx = np.random.choice(add_candidates)
    remove_idx = np.random.choice(remove_candidates)
    return int(remove_idx), int(add_idx)

def diversity_move(observation, profits, weights, capacities, archive_objs=None, best_observation=None):
    best_move = None
    best_min_dist = -np.inf
    items_selected = np.where(observation == 1)[0]
    items_unselected = np.where(observation == 0)[0]
    if len(items_selected) == 0 or len(items_unselected) == 0 or archive_objs is None:
        return mutation_move(observation, profits, weights, capacities)
    for _ in range(3):
        remove_idx = int(np.random.choice(items_selected))
        add_idx = int(np.random.choice(items_unselected))
        candidate = observation.copy()
        candidate[remove_idx] = 0
        candidate[add_idx] = 1
        candidate_objs = np.sum(profits * candidate, axis=1)
        dists = np.linalg.norm(archive_objs - candidate_objs, axis=1)
        min_dist = np.min(dists)
        if min_dist > best_min_dist:
            best_min_dist = min_dist
            best_move = (remove_idx, add_idx)
    return best_move if best_move is not None else (0, 0)

# Registry for moves
MOVE_REGISTRY = {
    "mutation": mutation_move,
    "local_search": local_search_move,
    "global_search": global_search_move,
    "follow": follow_move,
    "diversity": diversity_move,
}

In [5]:
class LLMFunctionGenerator:
    """Class for generating and evolving heuristic functions using LLM."""
    
    def __init__(self):
        self.function_cache = {}
        self.generation_count = 0
        self.max_generations = 5  # Reduced for performance
        self.pre_generated_functions = {}
        
    def pre_generate_functions(self):
        """Pre-generate a diverse set of functions before optimization starts"""
        move_types = ["mutation", "local_search", "global_search", "follow", "diversity"]
        for move_type in move_types:
            for i in range(2):  # Generate 2 variants per move type
                code = self.generate_sub_function(move_type)
                if code:
                    self.pre_generated_functions[f"{move_type}_{i}"] = code
        print(f"Pre-generated {len(self.pre_generated_functions)} functions")
    
    def generate_sub_function(self, function_type, context=None):
        """Generate a new sub-function using LLM."""
        if self.generation_count >= self.max_generations:
            return MOVE_REGISTRY.get(function_type, mutation_move)
        
        cache_key = f"sub_function_{function_type}_{self.generation_count}"
        if cache_key in self.function_cache:
            return self.function_cache[cache_key]
        
        try:
            user_prompt = f"Design a Python function for {function_type} move in a multi-objective knapsack problem."
            response = generate_llm_response("You are an expert in optimization heuristics.", user_prompt)
            code = extract_code_from_response(response)
            self.generation_count += 1
            
            # Validate the function
            input_args = [('array', 250), ('array_2d', (2, 250)), ('array_2d', (2, 250)), ('array', 2), None, None]
            is_valid, message = validate_function(code, f"{function_type}_move", input_args)
            
            if not is_valid:
                print(f"Generated function is invalid: {message}")
                return MOVE_REGISTRY.get(function_type, mutation_move)
            
            # Store in cache
            self.function_cache[cache_key] = code
            return code
        except Exception as e:
            print(f"Error generating function: {e}")
            return MOVE_REGISTRY.get(function_type, mutation_move)
    
    def crossover_evolution(self, parent1_code, parent2_code, function_type):
        """Perform crossover evolution between two parent functions."""
        return parent1_code  # Simplified for now
    
    def cooperative_evolution(self, sub_function_code, arch_function_code):
        """Perform cooperative evolution between functions."""
        return sub_function_code, arch_function_code  # Simplified for now
    
    def architecture_upgrade(self, arch_function_code, performance_feedback):
        """Upgrade the architecture function based on performance feedback."""
        return arch_function_code  # Simplified for now

In [6]:
class EnhancedAgentArchitecture:
    """Enhanced architecture function with LLM integration for evolution."""
    
    def __init__(self, move_types, smoothing=0.1, llm_generator=None):
        self.move_types = list(move_types)
        self.move_counts = {k: 1 for k in self.move_types}
        self.move_successes = {k: 1 for k in self.move_types}
        self.move_probs = np.ones(len(self.move_types)) / len(self.move_types)
        self.smoothing = smoothing
        self.llm_generator = llm_generator
        self.performance_history = []
        self.upgrade_count = 0
        
    def select_move(self):
        if len(self.move_types) == 0:
            return "mutation"
        idx = np.random.choice(len(self.move_types), p=self.move_probs)
        return self.move_types[idx]
    
    def report_move_result(self, move_type, success):
        if move_type not in self.move_types:
            return
        if move_type not in self.move_counts:
            self.move_counts[move_type] = 1
        if move_type not in self.move_successes:
            self.move_successes[move_type] = 1
            
        self.move_counts[move_type] += 1
        if success:
            self.move_successes[move_type] += 1
        
        # Update probabilities
        rates = np.array([self.move_successes[k]/self.move_counts[k] for k in self.move_types], dtype=np.float64)
        rates = np.clip(rates, 1e-8, 1.0)  # Prevent numerical issues
        exp_rates = np.exp(rates / self.smoothing)
        new_probs = exp_rates / np.sum(exp_rates)
        self.move_probs = 0.5 * self.move_probs + 0.5 * new_probs
        self.move_probs = self.move_probs / np.sum(self.move_probs)
        
        self.performance_history.append({
            'move_type': move_type,
            'success': success,
            'rates': rates.copy(),
            'probs': self.move_probs.copy()
        })
    
    def clone(self):
        arch = EnhancedAgentArchitecture(self.move_types, self.smoothing, self.llm_generator)
        arch.move_counts = dict(self.move_counts)
        arch.move_successes = dict(self.move_successes)
        arch.move_probs = np.copy(self.move_probs)
        arch.performance_history = list(self.performance_history)
        return arch
    
    def mutate(self, registry, mutation_rate=0.25):
        move_types = set(self.move_types)
        all_types = set(registry.keys())
        
        # Add new move type
        if random.random() < mutation_rate and len(move_types) < len(all_types):
            available_types = all_types - move_types
            if available_types:
                add = random.choice(list(available_types))
                move_types.add(add)
        
        # Remove poor performing move type
        if random.random() < mutation_rate and len(move_types) > 2:
            # Only consider move types that exist in our counts
            valid_move_types = [k for k in move_types if k in self.move_counts and k in self.move_successes]
            if len(valid_move_types) > 2:
                success_rates = {k: self.move_successes[k]/max(self.move_counts[k], 1) for k in valid_move_types}
                worst_move = min(success_rates, key=success_rates.get)
                if success_rates[worst_move] < 0.2:  # Only remove if really poor
                    move_types.remove(worst_move)
        
        self.move_types = list(move_types)
        random.shuffle(self.move_types)
        
        # Ensure all move types have entries in counts and successes
        for move_type in self.move_types:
            if move_type not in self.move_counts:
                self.move_counts[move_type] = 1
            if move_type not in self.move_successes:
                self.move_successes[move_type] = 1
        
        # Remove entries for move types no longer in use
        old_counts = dict(self.move_counts)
        old_successes = dict(self.move_successes)
        self.move_counts = {k: old_counts.get(k, 1) for k in self.move_types}
        self.move_successes = {k: old_successes.get(k, 1) for k in self.move_types}
        
        # Recalculate probabilities
        if len(self.move_types) > 0:
            self.move_probs = np.ones(len(self.move_types)) / len(self.move_types)
        else:
            self.move_types = ["mutation"]
            self.move_counts = {"mutation": 1}
            self.move_successes = {"mutation": 1}
            self.move_probs = np.array([1.0])

In [7]:
class EnhancedModularMetaAgent:
    """Enhanced modular agent with improved adaptation and optional LLM integration."""
    
    def __init__(self, num_items, num_objectives, profits, weights, capacities, 
                 move_types=None, smoothing=0.1, llm_generator=None):
        self.num_items = num_items
        self.num_objectives = num_objectives
        self.profits = profits
        self.weights = weights
        self.capacities = capacities
        self.llm_generator = llm_generator
        self.custom_moves = {}
        self.evolution_count = 0
        self.max_evolution_count = 3  # Limit LLM calls
        
        self.architecture = EnhancedAgentArchitecture(
            move_types if move_types else list(MOVE_REGISTRY.keys()), 
            smoothing=smoothing,
            llm_generator=llm_generator
        )
        
        self.best_observation = None
        self.last_move_type = None
        self.performance_history = []
        self.fitness_history = []
    
    def observe(self, state):
        return np.array(state["Items"], dtype=np.int32)
    
    def act(self, observation, context=None):
        archive_objs = context.get("archive_objs") if context else None
        best_obs = self.best_observation
        
        # Try to get a feasible move
        for attempt in range(3):
            move_type = self.architecture.select_move()
            
            # Select move function
            if move_type in self.custom_moves:
                move_fn = self.custom_moves[move_type]
            elif move_type in MOVE_REGISTRY:
                move_fn = MOVE_REGISTRY[move_type]
            else:
                # Fallback to mutation if unknown move type
                move_fn = MOVE_REGISTRY["mutation"]
                move_type = "mutation"
            
            try:
                move = move_fn(
                    observation, self.profits, self.weights, self.capacities,
                    archive_objs=archive_objs, best_observation=best_obs
                )
                
                if self._is_feasible(observation, move):
                    self.last_move_type = move_type
                    return move
                    
            except Exception as e:
                print(f"Error executing move {move_type}: {str(e)}")
        
        # Final fallback
        move = mutation_move(observation, self.profits, self.weights, self.capacities)
        self.last_move_type = "mutation"
        return move
    
    def report_move_result(self, move_type, success, fitness=None):
        if move_type is None:
            move_type = self.last_move_type
            
        self.architecture.report_move_result(move_type, success)
        
        self.performance_history.append({
            'move_type': move_type,
            'success': success,
            'timestamp': time.time()
        })
        
        if fitness is not None:
            self.fitness_history.append(fitness)
            # Keep only recent history for memory efficiency
            if len(self.fitness_history) > 100:
                self.fitness_history = self.fitness_history[-50:]
    
    def update_best(self, observation):
        self.best_observation = observation.copy()
    
    def _is_feasible(self, observation, move):
        if self.weights is None or self.capacities is None or move is None:
            return True
        
        try:
            remove_idx, add_idx = move
            
            # Basic checks
            if (remove_idx == add_idx or 
                remove_idx < 0 or remove_idx >= self.num_items or
                add_idx < 0 or add_idx >= self.num_items):
                return False
            
            # Check current state
            if observation[remove_idx] == 0 or observation[add_idx] == 1:
                return False
            
            # Check capacity constraints
            items = observation.copy()
            items[remove_idx] = 0
            items[add_idx] = 1
            
            total_weights = np.sum(self.weights * items, axis=1)
            return np.all(total_weights <= self.capacities)
            
        except (ValueError, IndexError, TypeError):
            return False
    
    def clone(self):
        agent = EnhancedModularMetaAgent(
            self.num_items, self.num_objectives, self.profits, self.weights, self.capacities,
            move_types=self.architecture.move_types,
            smoothing=self.architecture.smoothing,
            llm_generator=self.llm_generator
        )
        
        agent.architecture = self.architecture.clone()
        agent.custom_moves = dict(self.custom_moves)
        agent.evolution_count = self.evolution_count
        agent.max_evolution_count = self.max_evolution_count
        
        if self.best_observation is not None:
            agent.best_observation = self.best_observation.copy()
        
        # Copy recent performance history
        agent.performance_history = list(self.performance_history[-20:])
        agent.fitness_history = list(self.fitness_history[-20:])
        
        return agent
    
    def mutate(self, registry, mutation_rate=0.25):
        self.architecture.mutate(registry, mutation_rate=mutation_rate)
        
        # Occasionally try to generate new custom moves (if LLM available)
        if (ENABLE_LLM and self.llm_generator and 
            random.random() < 0.1 and 
            self.evolution_count < self.max_evolution_count):
            
            move_type = random.choice(self.architecture.move_types)
            try:
                code = self.llm_generator.generate_sub_function(move_type)
                if code:
                    namespace = {'np': np, 'random': random}
                    exec(code, namespace)
                    self.custom_moves[move_type] = namespace[f"{move_type}_move"]
                    self.evolution_count += 1
                    print(f"Generated new {move_type} move for agent")
            except Exception as e:
                print(f"Failed to generate new move function: {str(e)}")
    
    def get_performance_stats(self):
        """Get performance statistics for this agent"""
        if not self.performance_history:
            return {"success_rate": 0.0, "move_diversity": 0, "avg_fitness": 0.0}
        
        recent_history = self.performance_history[-50:]  # Last 50 moves
        success_rate = sum(1 for h in recent_history if h['success']) / len(recent_history)
        move_types = set(h['move_type'] for h in recent_history)
        move_diversity = len(move_types)
        avg_fitness = np.mean(self.fitness_history[-20:]) if self.fitness_history else 0.0
        
        return {
            "success_rate": success_rate,
            "move_diversity": move_diversity,
            "avg_fitness": avg_fitness,
            "total_moves": len(self.performance_history),
            "evolution_count": self.evolution_count
        }

In [8]:
class EnhancedAgentPopulationManager:
    """Enhanced population manager with LLM-based GSES cycle."""
    
    def __init__(self, num_agents, num_items, num_objectives, profits, weights, capacities, llm_generator=None):
        self.num_agents = num_agents
        self.llm_generator = llm_generator
        self.registry = MOVE_REGISTRY
        self.evolution_count = 0
        
        self.agents = [
            EnhancedModularMetaAgent(
                num_items, num_objectives, profits, weights, capacities,
                move_types=random.sample(list(MOVE_REGISTRY.keys()), k=random.randint(2, len(MOVE_REGISTRY))),
                llm_generator=llm_generator
            )
            for _ in range(num_agents)
        ]
        
        self.agent_stats = [{"fitness_sum": 0.0, "moves": 0} for _ in range(num_agents)]
        self.generation = 0
        self.performance_history = []
    
    def select_agent_for_solution(self, solution_idx):
        return self.agents[solution_idx % self.num_agents]
    
    def report_agent_performance(self, agent_idx, solution_fitness):
        if 0 <= agent_idx < len(self.agent_stats):
            self.agent_stats[agent_idx]["fitness_sum"] += solution_fitness
            self.agent_stats[agent_idx]["moves"] += 1
    
    def gses_agent_level(self):
        """Perform the GSES cycle at the agent level."""
        avg_fitness = []
        for s in self.agent_stats:
            if s["moves"] > 0:
                avg_fitness.append(s["fitness_sum"] / s["moves"])
            else:
                avg_fitness.append(-1e9)
        
        sorted_idx = np.argsort(-np.array(avg_fitness))
        survivors = [self.agents[i] for i in sorted_idx[:max(2, self.num_agents//2)]]
        
        self.performance_history.append({
            'generation': self.generation,
            'best_fitness': max(avg_fitness),
            'avg_fitness': np.mean(avg_fitness),
            'num_survivors': len(survivors)
        })
        
        new_agents = []
        while len(new_agents) + len(survivors) < self.num_agents:
            if random.random() < 0.7 and len(survivors) >= 2:
                # Crossover
                parent1, parent2 = random.sample(survivors, 2)
                child = parent1.clone()
                
                # Simple crossover: combine move types
                moves1 = set(parent1.architecture.move_types)
                moves2 = set(parent2.architecture.move_types)
                moves_child = list(moves1 | moves2)
                
                if len(moves_child) > 2:
                    moves_child = random.sample(moves_child, k=random.randint(2, len(moves_child)))
                
                child.architecture.move_types = moves_child
                child.architecture.move_counts = {k: 1 for k in moves_child}
                child.architecture.move_successes = {k: 1 for k in moves_child}
                child.architecture.move_probs = np.ones(len(moves_child)) / len(moves_child)
                
                child.mutate(self.registry, mutation_rate=0.25)
                new_agents.append(child)
            else:
                # Mutation only
                parent = random.choice(survivors)
                child = parent.clone()
                child.mutate(self.registry, mutation_rate=0.5)
                new_agents.append(child)
        
        self.agents = survivors + new_agents
        self.agent_stats = [{"fitness_sum": 0.0, "moves": 0} for _ in range(self.num_agents)]
        self.generation += 1
        
        if avg_fitness:
            print(f"Generation {self.generation}: Best fitness = {max(avg_fitness):.2f}, Avg fitness = {np.mean(avg_fitness):.2f}")
    
    def evolve_with_llm(self):
        """Perform LLM-based evolution on the population."""
        # Simplified for now - just call regular GSES
        pass

In [9]:
%%cython
from libc.stdlib cimport malloc, free, srand, rand
from libc.string cimport memset
from libc.math cimport exp
import numpy as np
cimport numpy as np
import time

cdef struct ind:
    int nombr_nonpris
    int nombr
    int rank
    float fitnessbest
    float fitness
    int explored
    double *f
    double *capa
    double *v
    int *d
    int *Items

cdef struct pop:
    int size
    int maxsize
    ind **ind_array

cdef int NBITEMS = 250
cdef int ni = 250
cdef int L = 5
cdef double LARGE = 10e50
cdef float smallValue = 0.0000001
cdef double kappa = 0.05
cdef int alpha = 10
cdef int paretoIni = 28000
cdef int nf = 2
cdef double *capacities = NULL
cdef int **weights = NULL
cdef int **profits = NULL
cdef double *vector_weight = NULL
cdef double max_bound = 0.0
cdef double **OBJ_Weights = NULL
cdef int nombreLIGNE = 0
cdef int nextLn = 0
cdef int inv = 0
cdef int OBJ_Weights_lines = 0

cdef public object py_agent_manager = None
cdef public object move_stats = None

def set_agent_manager(agent_manager):
    global py_agent_manager
    py_agent_manager = agent_manager

def reset_move_stats():
    global move_stats
    move_stats = {"total_agent_calls": 0, "move_type_counts": {}, "total_time_agent": 0.0, "total_time_loop": 0.0}

def print_move_stats():
    print("\n=== AGENT MOVE PROFILE ===")
    print("Total agent calls:", move_stats["total_agent_calls"])
    print("Move type counts:", move_stats["move_type_counts"])
    print("Total agent time (sec):", move_stats["total_time_agent"])
    print("Total loop time (sec):", move_stats["total_time_loop"])
    if move_stats["total_agent_calls"] > 0:
        print("Avg agent call time (ms):", 1000.0 * move_stats["total_time_agent"] / move_stats["total_agent_calls"])

def seed(int x):
    srand(x)

cdef int irand(int range_val):
    return rand() % range_val

cdef void *chk_malloc(size_t size):
    cdef void *return_value = malloc(size)
    if return_value == NULL:
        raise MemoryError("Out of memory.")
    memset(return_value, 0, size)
    return return_value

cdef pop *create_pop(int maxsize, int nf):
    cdef int i
    cdef pop *pp = <pop *>chk_malloc(sizeof(pop))
    pp.size = 0
    pp.maxsize = maxsize
    pp.ind_array = <ind **>chk_malloc(maxsize * sizeof(void*))
    for i in range(maxsize):
        pp.ind_array[i] = NULL
    return pp

cdef ind *create_ind(int nf):
    cdef int i
    cdef ind *p_ind = <ind *>chk_malloc(sizeof(ind))
    p_ind.nombr_nonpris = 0
    p_ind.nombr = 0
    p_ind.rank = 0
    p_ind.fitnessbest = -1.0
    p_ind.fitness = -1.0
    p_ind.explored = 0
    p_ind.f = <double *>chk_malloc(nf * sizeof(double))
    p_ind.capa = <double *>chk_malloc(nf * sizeof(double))
    p_ind.v = <double *>chk_malloc(nf * sizeof(double))
    p_ind.d = <int *>chk_malloc(ni * sizeof(int))
    p_ind.Items = <int *>chk_malloc(ni * sizeof(int))
    for i in range(ni):
        p_ind.Items[i] = 0
        p_ind.d[i] = 0
    for i in range(nf):
        p_ind.f[i] = 0.0
        p_ind.capa[i] = 0.0
        p_ind.v[i] = 0.0
    return p_ind

cdef ind *ind_copy(ind *i):
    cdef ind *p_ind = create_ind(nf)
    cdef int k
    p_ind.nombr_nonpris = i.nombr_nonpris
    p_ind.nombr = i.nombr
    p_ind.rank = i.rank
    p_ind.fitnessbest = i.fitnessbest
    p_ind.fitness = i.fitness
    p_ind.explored = i.explored
    for k in range(nf):
        p_ind.f[k] = i.f[k]
        p_ind.v[k] = i.v[k]
        p_ind.capa[k] = i.capa[k]
    for k in range(ni):
        p_ind.d[k] = i.d[k]
        p_ind.Items[k] = i.Items[k]
    return p_ind

cdef void free_ind(ind *p_ind):
    if p_ind != NULL:
        free(p_ind.d)
        free(p_ind.f)
        free(p_ind.capa)
        free(p_ind.v)
        free(p_ind.Items)
        free(p_ind)

cdef void complete_free_pop(pop *pp):
    cdef int i
    if pp != NULL:
        if pp.ind_array != NULL:
            for i in range(pp.size):
                if pp.ind_array[i] != NULL:
                    free_ind(pp.ind_array[i])
                    pp.ind_array[i] = NULL
            free(pp.ind_array)
        free(pp)

cdef void cleanup_globals():
    global capacities, weights, profits, vector_weight, OBJ_Weights, OBJ_Weights_lines, nf, ni
    if capacities != NULL:
        free(capacities)
        capacities = NULL
    if weights != NULL:
        for i in range(nf):
            if weights[i] != NULL:
                free(weights[i])
        free(weights)
        weights = NULL
    if profits != NULL:
        for i in range(nf):
            if profits[i] != NULL:
                free(profits[i])
        free(profits)
        profits = NULL
    if vector_weight != NULL:
        free(vector_weight)
        vector_weight = NULL
    if OBJ_Weights != NULL:
        for i in range(nf):
            if OBJ_Weights[i] != NULL:
                free(OBJ_Weights[i])
        free(OBJ_Weights)
        OBJ_Weights = NULL
    OBJ_Weights_lines = 0
    nf = 0
    ni = 0

cdef int non_dominated(ind *p_ind_a, ind *p_ind_b):
    cdef int i
    cdef int a_is_good = -1
    cdef int equal = 1
    for i in range(nf):
        if p_ind_a.f[i] > p_ind_b.f[i]:
            a_is_good = 1
        if p_ind_a.f[i] != p_ind_b.f[i]:
            equal = 0
    if equal:
        return 0
    return a_is_good

cdef double calcAddEpsIndicator(ind *p_ind_a, ind *p_ind_b):
    global max_bound
    cdef int i
    cdef double eps
    cdef double temp_eps
    if max_bound == 0.0:
        max_bound = 1e-8
    eps = (p_ind_a.v[0]/max_bound)-(p_ind_b.v[0]/max_bound)
    for i in range(1, nf):
        temp_eps = (p_ind_a.v[i]/max_bound)-(p_ind_b.v[i]/max_bound)
        if temp_eps > eps:
            eps = temp_eps
    return eps

cdef void init_fitness(ind *x):
    x.fitness = 0.0

cdef void update_fitness(ind *x, double I):
    x.fitness -= exp(-I / kappa)

cdef double update_fitness_return(double f, double I):
    return f - exp(-I / kappa)

cdef int delete_fitness(ind *x, double I):
    x.fitness += exp(-I / kappa)
    return 0

cdef void compute_ind_fitness(ind *x, pop *SP):
    cdef int j
    init_fitness(x)
    for j in range(SP.size):
        if SP.ind_array[j] != x:
            update_fitness(x, calcAddEpsIndicator(SP.ind_array[j], x))

cdef void compute_all_fitness(pop *SP):
    cdef int i
    for i in range(SP.size):
        compute_ind_fitness(SP.ind_array[i], SP)

cdef void loadMOKP(char *filename):
    global nf, ni, capacities, weights, profits
    cdef int i, f
    with open(filename.decode(), "r") as source:
        _nf, _ni = [int(x) for x in source.readline().split()]
        nf = _nf
        ni = _ni
        capacities = <double *>chk_malloc(nf * sizeof(double))
        weights = <int **>chk_malloc(nf * sizeof(void*))
        profits = <int **>chk_malloc(nf * sizeof(void*))
        for f in range(nf):
            capacities[f] = float(source.readline().strip())
            weights[f] = <int *>chk_malloc(ni * sizeof(int))
            profits[f] = <int *>chk_malloc(ni * sizeof(int))
            for i in range(ni):
                source.readline()
                weights[f][i] = int(source.readline().strip())
                profits[f][i] = int(source.readline().strip())

cdef void read_weights_file(char *filename):
    global OBJ_Weights, nombreLIGNE, nf, OBJ_Weights_lines
    cdef int i, j, nlines
    with open(filename.decode(), "r") as f:
        lines = [line for line in f if line.strip()]
    nlines = len(lines)
    OBJ_Weights = <double **>chk_malloc(nf * sizeof(void*))
    for i in range(nf):
        OBJ_Weights[i] = <double *>chk_malloc(nlines * sizeof(double))
    for i, line in enumerate(lines):
        vals = line.strip().split()
        for j in range(nf):
            OBJ_Weights[j][i] = float(vals[j])
    nombreLIGNE = nlines - 1
    OBJ_Weights_lines = nlines

cdef void dynamic_weight_allpop():
    global vector_weight, OBJ_Weights, nombreLIGNE, nf, nextLn
    cdef int i
    if vector_weight == NULL:
        vector_weight = <double *>chk_malloc(nf * sizeof(double))
    for i in range(nf):
        vector_weight[i] = OBJ_Weights[i][nextLn]
    if nextLn == nombreLIGNE:
        nextLn = 0
    else:
        nextLn += 1

cdef void choose_weight():
    dynamic_weight_allpop()

cdef void random_init_ind(ind *x):
    cdef int j, r, tmp
    for j in range(ni):
        x.d[j] = j
    for j in range(ni):
        r = irand(ni)
        tmp = x.d[r]
        x.d[r] = x.d[j]
        x.d[j] = tmp

cdef void evaluate(ind *x):
    cdef int j, l, k, faisable
    x.nombr = 0
    x.nombr_nonpris = 0
    for j in range(nf):
        x.capa[j] = 0.0
        x.f[j] = 0.0
    for j in range(ni):
        l = 0
        faisable = 1
        while l < nf and faisable == 1:
            if x.capa[l] + weights[l][x.d[j]] > capacities[l]:
                faisable = 0
            l += 1
        if faisable == 1:
            for k in range(nf):
                x.capa[k] += weights[k][x.d[j]]
                x.f[k] += profits[k][x.d[j]]
            x.Items[x.d[j]] = 1
            x.nombr += 1
        else:
            x.Items[x.d[j]] = 0
            x.nombr_nonpris += 1

cdef void P_init_pop(pop *SP, pop *Sarchive, int alpha):
    cdef int i, x, tmp, t
    t = max(alpha, Sarchive.size)
    cdef int* shuffle = <int *>chk_malloc(t * sizeof(int))
    for i in range(t):
        shuffle[i] = i
    for i in range(t):
        x = irand(alpha)
        tmp = shuffle[i]
        shuffle[i] = shuffle[x]
        shuffle[x] = tmp
    SP.size = alpha
    if Sarchive.size > alpha:
        for i in range(alpha):
            SP.ind_array[i] = ind_copy(Sarchive.ind_array[shuffle[i]])
    else:
        for i in range(alpha):
            if shuffle[i] < Sarchive.size:
                SP.ind_array[i] = ind_copy(Sarchive.ind_array[shuffle[i]])
            else:
                SP.ind_array[i] = create_ind(nf)
                random_init_ind(SP.ind_array[i])
                evaluate(SP.ind_array[i])
    free(shuffle)

cdef int extractPtoArchive(pop *P, pop *archive):
    cdef int i, j, dom, t, convergence_rate
    t = archive.size + P.size
    archiveAndP = create_pop(t, nf)
    convergence_rate = 0
    for i in range(archive.size):
        archiveAndP.ind_array[i] = archive.ind_array[i]
    for i in range(P.size):
        archiveAndP.ind_array[i + archive.size] = ind_copy(P.ind_array[i])
    archiveAndP.size = t
    archive.size = 0
    for i in range(t):
        for j in range(t):
            if i != j:
                dom = non_dominated(archiveAndP.ind_array[i], archiveAndP.ind_array[j])
                if dom == -1 or (dom == 0 and i > j):
                    break
        else:
            archive.ind_array[archive.size] = ind_copy(archiveAndP.ind_array[i])
            archive.size += 1
            if i >= t - P.size:
                convergence_rate += 1
    complete_free_pop(archiveAndP)
    return convergence_rate

cdef double calcMaxbound(pop *SP, int size):
    global max_bound
    cdef int i, j
    SP.size = size
    cdef double max_b = SP.ind_array[0].v[0]
    for i in range(SP.size):
        for j in range(nf):
            if max_b < SP.ind_array[i].v[j]:
                max_b = SP.ind_array[i].v[j]
    if max_b == 0.0:
        max_b = 1e-8
    max_bound = max_b
    return max_b

cdef void calcul_weight(pop *SP, int size):
    cdef int i, j
    for i in range(SP.size):
        for j in range(nf):
            SP.ind_array[i].v[j] = SP.ind_array[i].f[j] * vector_weight[j]

cdef int compute_fitness_and_select(pop *SP, ind *x, int size):
    cdef int i, worst
    cdef double worst_fit, fit_tmp
    SP.size = size
    x.fitness = 0
    compute_ind_fitness(x, SP)
    worst_fit = x.fitness
    worst = -1
    for i in range(SP.size):
        fit_tmp = update_fitness_return(SP.ind_array[i].fitness, calcAddEpsIndicator(x, SP.ind_array[i]))
        if fit_tmp > worst_fit:
            worst = i
            worst_fit = fit_tmp
    fit_tmp = x.fitness
    if worst == -1:
        return -1
    else:
        for i in range(SP.size):
            delete_fitness(SP.ind_array[i], calcAddEpsIndicator(SP.ind_array[worst], SP.ind_array[i]))
            update_fitness(SP.ind_array[i], calcAddEpsIndicator(x, SP.ind_array[i]))
        delete_fitness(x, calcAddEpsIndicator(SP.ind_array[worst], x))
        free_ind(SP.ind_array[worst])
        SP.ind_array[worst] = ind_copy(x)
        if fit_tmp - worst_fit > smallValue:
            return worst
        else:
            return -1

cdef np.ndarray get_archive_objs(pop *archive):
    cdef int i, j
    arr = np.zeros((archive.size, nf), dtype=np.float64)
    for i in range(archive.size):
        for j in range(nf):
            arr[i, j] = archive.ind_array[i].f[j]
    return arr

cdef np.ndarray get_items(ind *x):
    arr = np.zeros(ni, dtype=np.int32)
    cdef int i
    for i in range(ni):
        arr[i] = x.Items[i]
    return arr

cdef tuple cy_mutation_move(np.ndarray[np.int32_t, ndim=1] items, int ni):
    cdef int n_selected = 0, n_unselected = 0, i
    for i in range(ni):
        if items[i] == 1: n_selected += 1
        else: n_unselected += 1
    if n_selected == 0 or n_unselected == 0:
        return (0, 0)
    sel = [i for i in range(ni) if items[i] == 1]
    unsel = [i for i in range(ni) if items[i] == 0]
    remove_idx = sel[irand(n_selected)]
    add_idx = unsel[irand(n_unselected)]
    return remove_idx, add_idx

cdef tuple cy_local_search_move(np.ndarray[np.int32_t, ndim=1] items, double[:, :] profits, int ni, int nf):
    cdef int i
    sel = [i for i in range(ni) if items[i] == 1]
    unsel = [i for i in range(ni) if items[i] == 0]
    if not sel or not unsel:
        return (0, 0)
    profs_selected = [sum([profits[f, idx] for f in range(nf)]) for idx in sel]
    profs_unselected = [sum([profits[f, idx] for f in range(nf)]) for idx in unsel]
    remove_idx = sel[np.argmin(profs_selected)]
    add_idx = unsel[np.argmax(profs_unselected)]
    return remove_idx, add_idx

cdef tuple cy_global_search_move(np.ndarray[np.int32_t, ndim=1] items, double[:, :] profits, int ni, int nf):
    cdef int i
    sel = [i for i in range(ni) if items[i] == 1]
    unsel = [i for i in range(ni) if items[i] == 0]
    if not sel or not unsel:
        return (0, 0)
    profs_selected = np.array([sum([profits[f, idx] for f in range(nf)]) for idx in sel], dtype=np.float64) + 1e-9
    profs_unselected = np.array([sum([profits[f, idx] for f in range(nf)]) for idx in unsel], dtype=np.float64) + 1e-9
    remove_idx = sel[np.random.choice(len(sel), p=profs_selected/np.sum(profs_selected))]
    add_idx = unsel[np.random.choice(len(unsel), p=profs_unselected/np.sum(profs_unselected))]
    return remove_idx, add_idx

def get_profits_np():
    global profits, nf, ni
    arr = np.zeros((nf, ni), dtype=np.float64)
    for f in range(nf):
        for i in range(ni):
            arr[f, i] = profits[f][i]
    return arr

cdef void Indicator_local_search1(pop *SP, pop *Sarchive, int size):
    cdef ind *x
    cdef ind *y
    cdef int i_idx, j_idx, r_idx, t_idx, k_idx, l_idx, v_idx, sol_idx, mino_idx, mp_idx, maxp_idx
    cdef int consistant, pos_idx, stop_idx, convergence, ii_idx, tmp_pris, tmp_nonpris, taille, feasible, tv_idx, IM_idx
    cdef int* remplace = <int *>chk_malloc(L * sizeof(int))
    cdef np.ndarray[np.int32_t, ndim=1] items_mv = np.empty(ni, dtype=np.int32)
    SP.size = size
    extractPtoArchive(SP, Sarchive)
    profits_np = get_profits_np()
    while True:
        convergence = 0
        archive_objs = get_archive_objs(Sarchive) if Sarchive.size > 0 else None
        loop_start_time = time.time()
        for i_idx in range(SP.size):
            if not SP.ind_array[i_idx].explored:
                x = ind_copy(SP.ind_array[i_idx])
                j_idx = 0
                while j_idx < x.nombr:
                    for l_idx in range(L):
                        remplace[l_idx] = 0
                    for ii_idx in range(ni):
                        items_mv[ii_idx] = x.Items[ii_idx]
                    
                    move_types = ["mutation", "local_search", "global_search", "follow", "diversity"]
                    mt_idx = irand(5)
                    move_type = move_types[mt_idx]
                    remove_idx = -1
                    add_idx = -1
                    agent_move_type = move_type
                    agent_success = False
                    agent_start_time = time.time()
                    
                    if move_type == "mutation":
                        remove_idx, add_idx = cy_mutation_move(items_mv, ni)
                    elif move_type == "local_search":
                        remove_idx, add_idx = cy_local_search_move(items_mv, profits_np, ni, nf)
                    elif move_type == "global_search":
                        remove_idx, add_idx = cy_global_search_move(items_mv, profits_np, ni, nf)
                    elif (move_type == "diversity" or move_type == "follow") and py_agent_manager is not None:
                        agent_idx = i_idx % py_agent_manager.num_agents
                        agent = py_agent_manager.agents[agent_idx]
                        state = {
                            "Items": [x.Items[ii_idx] for ii_idx in range(ni)],
                            "capa": [x.capa[ii_idx] for ii_idx in range(nf)],
                            "f": [x.f[ii_idx] for ii_idx in range(nf)],
                        }
                        context = {}
                        if archive_objs is not None:
                            context["archive_objs"] = archive_objs
                        try:
                            obs = agent.observe(state)
                            remove_idx, add_idx = agent.act(obs, context)
                            if hasattr(agent, "last_move_type"):
                                agent_move_type = agent.last_move_type
                        except Exception as e:
                            print("Agent error:", e)
                            remove_idx = -1
                            add_idx = -1
                            agent_move_type = "error"
                    
                    agent_end_time = time.time()
                    if move_stats is not None:
                        move_stats["total_agent_calls"] += 1
                        move_stats["total_time_agent"] += (agent_end_time - agent_start_time)
                        mt = agent_move_type if agent_move_type is not None else "unknown"
                        if mt not in move_stats["move_type_counts"]:
                            move_stats["move_type_counts"][mt] = 0
                        move_stats["move_type_counts"][mt] += 1
                    
                    if remove_idx < 0 or remove_idx >= ni or x.Items[remove_idx] != 1:
                        while True:
                            mino_idx = irand(ni)
                            if x.Items[mino_idx] == 1:
                                break
                        remove_idx = mino_idx
                    if add_idx < 0 or add_idx >= ni or x.Items[add_idx] != 0 or add_idx == remove_idx:
                        while True:
                            maxp_idx = irand(ni)
                            if x.Items[maxp_idx] == 0 and maxp_idx != remove_idx:
                                break
                        add_idx = maxp_idx
                    
                    x.Items[remove_idx] = 0
                    x.nombr -= 1
                    x.nombr_nonpris += 1
                    for r_idx in range(nf):
                        x.capa[r_idx] -= weights[r_idx][remove_idx]
                        x.f[r_idx] -= profits[r_idx][remove_idx]
                    
                    IM_idx = 0
                    taille = 0
                    while IM_idx < L:
                        item_to_add = add_idx
                        if item_to_add < 0 or item_to_add >= ni or x.Items[item_to_add] != 0 or item_to_add == remove_idx:
                            while True:
                                item_to_add = irand(ni)
                                if x.Items[item_to_add] == 0 and item_to_add != remove_idx:
                                    break
                        consistant = 1
                        r_idx = 0
                        while r_idx < nf and consistant == 1:
                            if x.capa[r_idx] + weights[r_idx][item_to_add] > capacities[r_idx]:
                                consistant = 0
                            r_idx += 1
                        if consistant == 1:
                            feasible = 1
                            r_idx = 0
                            while r_idx < taille and feasible:
                                if item_to_add == remplace[r_idx]:
                                    feasible = 0
                                r_idx += 1
                            if feasible == 1:
                                remplace[taille] = item_to_add
                                taille += 1
                                x.Items[item_to_add] = 1
                                x.nombr_nonpris -= 1
                                x.nombr += 1
                                for r_idx in range(nf):
                                    x.capa[r_idx] += weights[r_idx][item_to_add]
                                    x.f[r_idx] += profits[r_idx][item_to_add]
                        IM_idx += 1
                    
                    for tv_idx in range(nf):
                        x.v[tv_idx] = x.f[tv_idx] * vector_weight[tv_idx]
                    
                    max_bound = calcMaxbound(SP, SP.size)
                    sol_idx = compute_fitness_and_select(SP, x, SP.size)
                    agent_success = (sol_idx != -1)
                    
                    if py_agent_manager is not None and (move_type == "diversity" or move_type == "follow"):
                        try:
                            agent.report_move_result(agent_move_type, agent_success)
                        except Exception:
                            pass
                    
                    if py_agent_manager is not None:
                        agent_idx = i_idx % py_agent_manager.num_agents
                        fitness = np.sum([x.f[j_idx] for j_idx in range(nf)])
                        py_agent_manager.report_agent_performance(agent_idx, fitness)
                    
                    if sol_idx != -1:
                        j_idx = x.nombr + 1
                        if sol_idx > i_idx and i_idx + 1 < SP.size:
                            y = SP.ind_array[i_idx + 1]
                            SP.ind_array[i_idx + 1] = SP.ind_array[sol_idx]
                            SP.ind_array[sol_idx] = y
                            i_idx += 1
                        break
                    elif sol_idx == -1:
                        x.Items[remove_idx] = 1
                        x.nombr_nonpris -= 1
                        x.nombr += 1
                        for r_idx in range(nf):
                            x.capa[r_idx] += weights[r_idx][remove_idx]
                            x.f[r_idx] += profits[r_idx][remove_idx]
                        if taille >= 1:
                            for r_idx in range(taille):
                                x.Items[remplace[r_idx]] = 0
                                x.nombr -= 1
                                x.nombr_nonpris += 1
                                for t_idx in range(nf):
                                    x.capa[t_idx] -= weights[t_idx][remplace[r_idx]]
                                    x.f[t_idx] -= profits[t_idx][remplace[r_idx]]
                                    x.v[t_idx] = x.f[t_idx] * vector_weight[t_idx]
                    j_idx += 1
                tmp_pris = x.nombr
                tmp_nonpris = x.nombr_nonpris
                free_ind(x)
                if j_idx == tmp_pris:
                    SP.ind_array[i_idx].explored = 1
        loop_end_time = time.time()
        if move_stats is not None:
            move_stats["total_time_loop"] += (loop_end_time - loop_start_time)
        convergence = extractPtoArchive(SP, Sarchive)
        if not convergence:
            break
    free(remplace)

def run_moacp(instance_file, weights_file, nbitems, num_objectives, output_file):
    global nf, ni, NBITEMS, alpha, paretoIni, L, nombreLIGNE, nextLn, inv, vector_weight
    global capacities, weights, profits, OBJ_Weights
    
    alpha = 10
    paretoIni = 28000
    NBL = 25
    NRUNS = 3
    
    reset_move_stats()
    
    for run in range(1, NRUNS+1):
        NBITEMS = nbitems
        ni = nbitems
        nf = num_objectives
        print(f"RUN {run}/{NRUNS} -- {instance_file.decode()} nbitems={ni} nf={nf} => {output_file}")
        nombreLIGNE = 0
        nextLn = 0
        inv = 0
        seed(run)
        loadMOKP(instance_file)
        read_weights_file(weights_file)
        vector_weight = <double *>chk_malloc(nf * sizeof(double))
        P = create_pop(paretoIni, nf)
        it = 0
        while it < NBL:
            solutions = create_pop(alpha, nf)
            archive = create_pop(paretoIni, nf)
            choose_weight()
            P_init_pop(solutions, P, alpha)
            extractPtoArchive(solutions, P)
            calcul_weight(solutions, alpha)
            calcMaxbound(solutions, alpha)
            compute_all_fitness(solutions)
            Indicator_local_search1(solutions, archive, alpha)
            extractPtoArchive(archive, P)
            
            if py_agent_manager is not None and hasattr(py_agent_manager, "gses_agent_level") and it % 10 == 0:
                py_agent_manager.gses_agent_level()
                if it % 20 == 0 and hasattr(py_agent_manager, "evolve_with_llm"):
                    py_agent_manager.evolve_with_llm()
            
            it += 1
            complete_free_pop(solutions)
            complete_free_pop(archive)
        
        with open(output_file, "a") as fpareto:
            fpareto.write("\n")
            for i in range(P.size):
                for j in range(nf):
                    fpareto.write(f"{P.ind_array[i].f[j]:.6f} ")
                fpareto.write("\n")
        
        complete_free_pop(P)
        cleanup_globals()
        print_move_stats()

In [10]:
# --- Parameters and File Names ---
nbitems = 250
num_objectives = 2
instance_file = b"250.2.txt"
weights_file = b"Weights_2obj_FQ200.txt"
output_file = "kss5_llm_enhanced.txt"
random_seed = 42

# 1. Load your instance as usual
profits = load_profits_from_file("250.2.txt", num_objectives, nbitems)
weights = load_weights_from_file("250.2.txt", num_objectives, nbitems)
capacities = load_capacities_from_file("250.2.txt", num_objectives, nbitems)

# 2. Create LLM function generator
llm_generator = LLMFunctionGenerator()

# 3. Pre-generate functions before optimization starts
print("Pre-generating functions...")
llm_generator.pre_generate_functions()

# 4. Create the enhanced modular MHRE agent population
# Use a smaller number of agents for faster execution
agent_manager = EnhancedAgentPopulationManager(
    num_agents=5,  # Reduced from 10 to 5
    num_items=nbitems,
    num_objectives=num_objectives,
    profits=profits,
    weights=weights,
    capacities=capacities,
    llm_generator=llm_generator
)

# 5. Register agent manager with cython
set_agent_manager(agent_manager)

# 6. Run the optimizer with reduced iterations for testing
print("Starting optimization...")
run_moacp(instance_file, weights_file, nbitems, num_objectives, output_file)
print("Optimization completed.")

Pre-generating functions...
Error generating LLM response: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)
Generated function is invalid: Function 'mutation_move' not found in the generated code.
Error generating LLM response: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)
Generated function is invalid: Function 'mutation_move' not found in the generated code.
Error generating LLM response: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)
Generated function is invalid: Function 'local_search_move' not found in the generated code.
Error generating LLM response: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)
Generated function is invalid: Function 'local_search_move' not found in the generated code.
Error generating LLM response: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=30)
Generated function is invalid: Fu

KeyError: 'mutation'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def load_pareto_front(filename):
    points = []
    with open(filename) as f:
        for line in f:
            line = line.strip()
            if line:
                points.append([float(x) for x in line.split()])
    return np.array(points)

def is_dominated(point, others):
    """Returns True if point is dominated by any row in others"""
    return np.any(np.all(others >= point, axis=1) & np.any(others > point, axis=1))

def get_pareto_front(points):
    mask = np.ones(len(points), dtype=bool)
    for i, p in enumerate(points):
        others = np.delete(points, i, axis=0)
        if is_dominated(p, others):
            mask[i] = False
    return points[mask]

# Load and merge all runs' points for each result file
ref_points = load_pareto_front("2502_Resulats.txt")
new_points = load_pareto_front("kss5_llm_enhanced.txt")

# Get the true Pareto front for each
ref_pareto = get_pareto_front(ref_points)
new_pareto = get_pareto_front(new_points)

# Plot
plt.figure(figsize=(10, 8))
plt.scatter(ref_pareto[:,0], ref_pareto[:,1], label="Reference True Pareto", alpha=0.7, color='blue')
plt.scatter(new_pareto[:,0], new_pareto[:,1], label="LLM-Enhanced True Pareto", alpha=0.7, color='red')
plt.xlabel("Objective 1")
plt.ylabel("Objective 2")
plt.legend()
plt.title("Comparison of Pareto Fronts: Reference vs LLM-Enhanced")
plt.grid(True)
plt.show()

# Calculate hypervolume improvement
def hypervolume(points, reference_point):
    """Calculate hypervolume indicator for a set of points."""
    sorted_points = sorted(points, key=lambda x: x[0])
    volume = 0.0
    prev_x = reference_point[0]
    for point in sorted_points:
        width = prev_x - point[0]
        height = reference_point[1] - point[1]
        volume += width * height
        prev_x = point[0]
    return volume

# Find reference point (max of each objective + 10%)
ref_point = np.max(np.vstack([ref_pareto, new_pareto]), axis=0) * 1.1

# Calculate hypervolumes
ref_hv = hypervolume(ref_pareto, ref_point)
new_hv = hypervolume(new_pareto, ref_point)

print(f"Reference Pareto Front Hypervolume: {ref_hv:.2f}")
print(f"LLM-Enhanced Pareto Front Hypervolume: {new_hv:.2f}")
print(f"Improvement: {(new_hv - ref_hv) / ref_hv * 100:.2f}%")

# Plot agent performance over generations
if hasattr(agent_manager, 'performance_history') and agent_manager.performance_history:
    generations = [p['generation'] for p in agent_manager.performance_history]
    best_fitness = [p['best_fitness'] for p in agent_manager.performance_history]
    avg_fitness = [p['avg_fitness'] for p in agent_manager.performance_history]
    
    plt.figure(figsize=(10, 6))
    plt.plot(generations, best_fitness, label='Best Fitness', marker='o')
    plt.plot(generations, avg_fitness, label='Average Fitness', marker='s')
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.title('Agent Population Performance Over Generations')
    plt.legend()
    plt.grid(True)
    plt.show()

In [15]:
!pip install scikit-learn


[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
A:\conda\envs\ibmols\python.exe

In [37]:
import sys
print(sys.executable)

A:\conda\envs\ibmols\python.exe


In [5]:
import sklearn
print(sklearn.__file__)

A:\conda\envs\ibmols\lib\site-packages\sklearn\__init__.py


In [5]:
import numpy, scipy, sklearn
print(numpy.__version__, scipy.__version__, sklearn.__version__)

1.26.4 1.13.1 1.6.1


In [6]:
import requests
resp = requests.post("http://localhost:11434/api/generate", json={"model": "llama3", "prompt": "def f(): return 1", "stream": False})
print(resp.json())

{'model': 'llama3', 'created_at': '2025-09-06T17:51:18.9589415Z', 'response': "A nice simple one!\n\nThe function `f` is defined as:\n```\ndef f(): return 1\n```\nThis is a function that takes no arguments and returns the value `1`.\n\nYou can call this function like any other:\n```\nresult = f()\nprint(result)  # Output: 1\n```\nNote that there's no parameter list (`()`), which means the function doesn't take any arguments. The `return` statement specifies what value is returned when the function is called. In this case, it's always `1`.", 'done': True, 'done_reason': 'stop', 'context': [128006, 882, 128007, 271, 755, 282, 4658, 471, 220, 16, 128009, 128006, 78191, 128007, 271, 32, 6555, 4382, 832, 2268, 791, 734, 1595, 69, 63, 374, 4613, 439, 512, 14196, 4077, 755, 282, 4658, 471, 220, 16, 198, 14196, 4077, 2028, 374, 264, 734, 430, 5097, 912, 6105, 323, 4780, 279, 907, 1595, 16, 63438, 2675, 649, 1650, 420, 734, 1093, 904, 1023, 512, 14196, 4077, 1407, 284, 282, 746, 1374, 4556, 8, 